# Map reconstruction with 6 inputs per pixel

In [ ]:
#libraries
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import GaussianDropout
from tensorflow.keras import optimizers
import tensorflow as tf

import os

In [ ]:
# load json and create model

json_file = open(r'./NN_qw_gen6opt.json', 'r')

loaded_model_json = json_file.read()

json_file.close()

loaded_model = tf.keras.models.model_from_json(loaded_model_json)

# load weights into new model

loaded_model.load_weights(r'./NN_qw_gen6opt.h5')

print("Loaded model from disk")

In [ ]:
#select the number of pixels
num_pix=73 #the map is num_pix*num_pix

In [ ]:
#select the folder in which the data is stored
data_folder='\\experimental_data_73(6)\\2'
#select the folder with the theoretical values (if needed)
th_folder='\\experimental_data_73(6)\\2\\values'

# Theoretical values (comment if unnecessary)

In [ ]:
file_names=[]
for file in os.listdir(os.getcwd()+th_folder):
    file_names+=[file]

In [ ]:
th_mat=np.zeros([num_pix,num_pix,4])

theta_th_mat=np.zeros([num_pix,num_pix])
nx_th_mat=np.zeros([num_pix,num_pix])
ny_th_mat=np.zeros([num_pix,num_pix])
nz_th_mat=np.zeros([num_pix,num_pix])

k=0
for Namefile in file_names:
    file=open(os.getcwd()+th_folder+'\\'+Namefile)
    
    file_row=[]
    
    
    for line in file:
        file_row += [line.split()]
    for i in range(num_pix):
        for j in range(num_pix):
            th_mat[i,j,k]=float(file_row[i][j])
    k+=1
    file.close()
    
theta_th_mat=th_mat[:,:,0]
nx_th_mat=th_mat[:,:,1]
ny_th_mat=th_mat[:,:,2]
nz_th_mat=th_mat[:,:,3]

# Experimental data

In [ ]:
#NB: the order for the data must be LL LH LD HL HH HD
file_names2=[]
for file in os.listdir(os.getcwd()+data_folder):
    file_names2+=[file]

In [ ]:
mat_data=np.zeros([num_pix,num_pix,6])
k=0
for Namefile in file_names2[:-1]: #select only the data that are needed in the right order!
    file=open(os.getcwd()+data_folder +'\\'+ Namefile)
    
    file_row=[]
    
    
    for line in file:
        file_row += [line.split()]
    for i in range(num_pix):
        for j in range(num_pix):
            mat_data[i,j,k]=float(file_row[i][j])
    k+=1
    file.close()

In [ ]:
flat_input=mat_data.reshape((num_pix*num_pix,6))

# NN prediction

In [ ]:
y_pred=loaded_model.predict(flat_input)
thetavect=y_pred[:,0]*np.pi
nx_vect=y_pred[:,1]*2 -1 
ny_vect=(y_pred[:,2]*2 -1)*np.sqrt(1-nx_vect**2)

nz_vect=np.sqrt(abs(1-nx_vect**2-ny_vect**2))

# Continuity and error correction

In [ ]:
#preliminary: continuity of the parameters between pixels 1 and 2

diff1=np.amax(np.array([abs(thetavect[1]-thetavect[0]),abs(nx_vect[1]-nx_vect[0]),abs(ny_vect[1]-ny_vect[0])]))
alt_vect=np.zeros([4])
alt_vect[0]=np.pi-thetavect[1]
alt_vect[1]=-nx_vect[1]
alt_vect[2]=-ny_vect[1]
alt_vect[3]=-nz_vect[1]
diff_alt_1=np.amax(np.array([abs(alt_vect[0]-thetavect[0]),abs(alt_vect[1]-nx_vect[0]),abs(alt_vect[2]-ny_vect[0])]))

if diff_alt_1<diff1:
    thetavect[1]=alt_vect[0]
    nx_vect[1]=alt_vect[1]
    ny_vect[1]=alt_vect[2]
    nz_vect[1]=alt_vect[3]

In [ ]:
#imposing continuity of the parameters to choose between U and -U

for i in range(2,num_pix**2):
    diff1=np.amax(np.array([abs(thetavect[i]-thetavect[i-1]),abs(nx_vect[i]-nx_vect[i-1]),abs(ny_vect[i]-ny_vect[i-1])]))
    diff2=np.amax(np.array([abs(thetavect[i]-thetavect[i-2]),abs(nx_vect[i]-nx_vect[i-2]),abs(ny_vect[i]-ny_vect[i-2])]))
    alt_vect=np.zeros([4])
    alt_vect[0]=np.pi-thetavect[i]
    alt_vect[1]=-nx_vect[i]
    alt_vect[2]=-ny_vect[i]
    alt_vect[3]=-nz_vect[i]
    diff_alt_1=np.amax(np.array([abs(alt_vect[0]-thetavect[i-1]),abs(alt_vect[1]-nx_vect[i-1]),abs(alt_vect[2]-ny_vect[i-1])]))
    diff_alt_2=np.amax(np.array([abs(alt_vect[0]-thetavect[i-2]),abs(alt_vect[1]-nx_vect[i-2]),abs(alt_vect[2]-ny_vect[i-2])]))
    if min(diff_alt_1,diff_alt_2)<min(diff1,diff2):
        thetavect[i]=alt_vect[0]
        nx_vect[i]=alt_vect[1]
        ny_vect[i]=alt_vect[2]
        nz_vect[i]=alt_vect[3]

In [ ]:
mat_theta=thetavect.reshape((num_pix,num_pix))
mat_nx=nx_vect.reshape((num_pix,num_pix))
mat_ny=ny_vect.reshape((num_pix,num_pix))
mat_nz=nz_vect.reshape((num_pix,num_pix))

In [ ]:
#minimum fidelity between neighbouring pixels for the 1st run
tol=0.9

In [ ]:
def fidelity(mat1,mat2):
    prod=np.trace(np.dot(np.conjugate(mat1.T),mat2))
    
    return 0.5*np.abs(prod)

def op_par(En,nx,ny,nz):
    mat=np.zeros([2,2],dtype=complex)
    
    mat[0,0]=np.cos(En) - 1j*np.sin(En)*nz
    mat[0,1]=-1j*np.sin(En)*(nx - 1j*ny)
    mat[1,0]=-1j*np.sin(En)*(nx + 1j*ny)
    mat[1,1]=np.cos(En) + 1j*np.sin(En)*nz
    
    return mat

In [ ]:
def close_fid(i,j):
    
    fids=np.zeros([8])
    
    fids[0]=fidelity(op_par(mat_theta[i%num_pix,j%num_pix],mat_nx[i%num_pix,j%num_pix],mat_ny[i%num_pix,j%num_pix],mat_nz[i%num_pix,j%num_pix]),op_par(mat_theta[(i-1)%num_pix,j%num_pix],mat_nx[(i-1)%num_pix,j%num_pix],mat_ny[(i-1)%num_pix,j%num_pix],mat_nz[(i-1)%num_pix,j%num_pix]))
    fids[1]=fidelity(op_par(mat_theta[i%num_pix,j%num_pix],mat_nx[i%num_pix,j%num_pix],mat_ny[i%num_pix,j%num_pix],mat_nz[i%num_pix,j%num_pix]),op_par(mat_theta[(i-1)%num_pix,(j-1)%num_pix],mat_nx[(i-1)%num_pix,(j-1)%num_pix],mat_ny[(i-1)%num_pix,(j-1)%num_pix],mat_nz[(i-1)%num_pix,(j-1)%num_pix]))
    fids[2]=fidelity(op_par(mat_theta[i%num_pix,j%num_pix],mat_nx[i%num_pix,j%num_pix],mat_ny[i%num_pix,j%num_pix],mat_nz[i%num_pix,j%num_pix]),op_par(mat_theta[i%num_pix,(j-1)%num_pix],mat_nx[i%num_pix,(j-1)%num_pix],mat_ny[i%num_pix,(j-1)%num_pix],mat_nz[i%num_pix,(j-1)%num_pix]))
    fids[3]=fidelity(op_par(mat_theta[i%num_pix,j%num_pix],mat_nx[i%num_pix,j%num_pix],mat_ny[i%num_pix,j%num_pix],mat_nz[i%num_pix,j%num_pix]),op_par(mat_theta[(i+1)%num_pix,(j-1)%num_pix],mat_nx[(i+1)%num_pix,(j-1)%num_pix],mat_ny[(i+1)%num_pix,(j-1)%num_pix],mat_nz[(i+1)%num_pix,(j-1)%num_pix]))
    fids[4]=fidelity(op_par(mat_theta[i%num_pix,j%num_pix],mat_nx[i%num_pix,j%num_pix],mat_ny[i%num_pix,j%num_pix],mat_nz[i%num_pix,j%num_pix]),op_par(mat_theta[(i+1)%num_pix,j%num_pix],mat_nx[(i+1)%num_pix,j%num_pix],mat_ny[(i+1)%num_pix,j%num_pix],mat_nz[(i+1)%num_pix,j%num_pix]))
    fids[5]=fidelity(op_par(mat_theta[i%num_pix,j%num_pix],mat_nx[i%num_pix,j%num_pix],mat_ny[i%num_pix,j%num_pix],mat_nz[i%num_pix,j%num_pix]),op_par(mat_theta[(i+1)%num_pix,(j+1)%num_pix],mat_nx[(i+1)%num_pix,(j+1)%num_pix],mat_ny[(i+1)%num_pix,(j+1)%num_pix],mat_nz[(i+1)%num_pix,(j+1)%num_pix]))
    fids[6]=fidelity(op_par(mat_theta[i%num_pix,j%num_pix],mat_nx[i%num_pix,j%num_pix],mat_ny[i%num_pix,j%num_pix],mat_nz[i%num_pix,j%num_pix]),op_par(mat_theta[i%num_pix,(j+1)%num_pix],mat_nx[i%num_pix,(j+1)%num_pix],mat_ny[i%num_pix,(j+1)%num_pix],mat_nz[i%num_pix,(j+1)%num_pix]))
    fids[7]=fidelity(op_par(mat_theta[i%num_pix,j%num_pix],mat_nx[i%num_pix,j%num_pix],mat_ny[i%num_pix,j%num_pix],mat_nz[i%num_pix,j%num_pix]),op_par(mat_theta[(i-1)%num_pix,(j+1)%num_pix],mat_nx[(i-1)%num_pix,(j+1)%num_pix],mat_ny[(i-1)%num_pix,(j+1)%num_pix],mat_nz[(i-1)%num_pix,(j+1)%num_pix]))
    
    fid_val=max(fids)
    
    return fid_val

In [ ]:
def inter_NN(i,j,mat_par):
    
    pars=np.zeros([8])
    trust=np.zeros([8])
    
    trust[0]=(close_fid(i-1,j)>tol)
    trust[1]=(close_fid(i-1,j-1)>tol)
    trust[2]=(close_fid(i,j-1)>tol)
    trust[3]=(close_fid(i+1,j-1)>tol)
    trust[4]=(close_fid(i+1,j)>tol)
    trust[5]=(close_fid(i+1,j+1)>tol)
    trust[6]=(close_fid(i,j+1)>tol)
    trust[7]=(close_fid(i-1,j+1)>tol)
    
    pars[0]=mat_par[(i-1)%num_pix,j]
    pars[1]=mat_par[(i-1)%num_pix,(j-1)%num_pix]
    pars[2]=mat_par[i,(j-1)%num_pix]
    pars[3]=mat_par[(i+1)%num_pix,(j-1)%num_pix]
    pars[4]=mat_par[(i+1)%num_pix,j]
    pars[5]=mat_par[(i+1)%num_pix,(j+1)%num_pix]
    pars[6]=mat_par[i,(j+1)%num_pix]
    pars[7]=mat_par[(i-1)%num_pix,(j+1)%num_pix]
    
    mean_par=sum(pars*trust)/sum(trust)
    
    return mean_par

In [ ]:
#correcting wrong pixels by interpolating between their neighbours

for i in range(num_pix):
    for j in range(num_pix):
    
        if close_fid(i,j)<tol:
            mat_theta[i,j]=inter_NN(i,j,mat_theta)
            mat_nx[i,j]=inter_NN(i,j,mat_nx)
            mat_ny[i,j]=inter_NN(i,j,mat_ny)
            mat_nz[i,j]=inter_NN(i,j,mat_nz)
            
            norm=np.sqrt(mat_nx[i,j]**2 + mat_ny[i,j]**2 + mat_nz[i,j]**2)
            
            mat_nx[i,j]=mat_nx[i,j]/norm
            mat_ny[i,j]=mat_ny[i,j]/norm
            mat_nz[i,j]=mat_nz[i,j]/norm

In [ ]:
alt_mat_theta=np.ones([num_pix,num_pix])*np.pi - mat_theta
alt_mat_nx=-mat_nx
alt_mat_ny=-mat_ny
alt_mat_nz=-mat_nz

In [ ]:
def max_par_jump(i,j,k,h):
    
    return np.amax(np.array([abs(mat_theta[i,j]-mat_theta[i-1,j-1])/np.pi,abs(mat_nx[i,j]-mat_nx[i-1,j-1])/2,abs(mat_ny[i,j]-mat_ny[i-1,j-1])/2]))

In [ ]:
def alt_max_par_jump(i,j,k,h):
    
    return np.amax(np.array([abs(alt_mat_theta[i,j]-mat_theta[i-1,j-1])/np.pi,abs(alt_mat_nx[i,j]-mat_nx[i-1,j-1])/2,abs(alt_mat_ny[i,j]-mat_ny[i-1,j-1])/2]))

In [ ]:
def close_jump(i,j):
    
    jumps=np.zeros([8])
    
    jumps[0]=max_par_jump(i,j,(i-1)%num_pix,j)
    jumps[1]=max_par_jump(i,j,(i-1)%num_pix,(j-1)%num_pix)
    jumps[2]=max_par_jump(i,j,i,(j-1)%num_pix)
    jumps[3]=max_par_jump(i,j,(i+1)%num_pix,(j-1)%num_pix)
    jumps[4]=max_par_jump(i,j,(i+1)%num_pix,j)
    jumps[5]=max_par_jump(i,j,(i+1)%num_pix,(j+1)%num_pix)
    jumps[6]=max_par_jump(i,j,i,(j+1)%num_pix)
    jumps[7]=max_par_jump(i,j,(i-1)%num_pix,(j+1)%num_pix)
    
    jump_val=min(jumps)
    
    return jump_val

In [ ]:
def alt_close_jump(i,j):
    
    jumps=np.zeros([8])
    
    jumps[0]=alt_max_par_jump(i,j,(i-1)%num_pix,j)
    jumps[1]=alt_max_par_jump(i,j,(i-1)%num_pix,(j-1)%num_pix)
    jumps[2]=alt_max_par_jump(i,j,i,(j-1)%num_pix)
    jumps[3]=alt_max_par_jump(i,j,(i+1)%num_pix,(j-1)%num_pix)
    jumps[4]=alt_max_par_jump(i,j,(i+1)%num_pix,j)
    jumps[5]=alt_max_par_jump(i,j,(i+1)%num_pix,(j+1)%num_pix)
    jumps[6]=alt_max_par_jump(i,j,i,(j+1)%num_pix)
    jumps[7]=alt_max_par_jump(i,j,(i-1)%num_pix,(j+1)%num_pix)
    
    jump_val=min(jumps)
    
    return jump_val

In [ ]:
#NN continuity correction (now in 2D)

for i in range(num_pix):
    for j in range(num_pix):
    
        if alt_close_jump(i,j)<close_jump(i,j):
            mat_theta[i,j]=alt_mat_theta[i,j]
            mat_nx[i,j]=alt_mat_nx[i,j]
            mat_ny[i,j]=alt_mat_ny[i,j]
            mat_nz[i,j]=alt_mat_nz[i,j]
            
            norm=np.sqrt(mat_nx[i,j]**2 + mat_ny[i,j]**2 + mat_nz[i,j]**2)
            
            mat_nx[i,j]=mat_nx[i,j]/norm
            mat_ny[i,j]=mat_ny[i,j]/norm
            mat_nz[i,j]=mat_nz[i,j]/norm

In [ ]:
thetavect2=mat_theta.reshape((num_pix*num_pix))
nx_vect2=mat_nx.reshape((num_pix*num_pix))
ny_vect2=mat_ny.reshape((num_pix*num_pix))
nz_vect2=mat_nz.reshape((num_pix*num_pix))

In [ ]:
#imposing continuity of the parameters to choose between U and -U (again)

for i in range(2,num_pix**2):
    diff1=np.amax(np.array([abs(thetavect2[i]-thetavect2[i-1]),abs(nx_vect2[i]-nx_vect2[i-1]),abs(ny_vect2[i]-ny_vect2[i-1])]))
    diff2=np.amax(np.array([abs(thetavect2[i]-thetavect2[i-2]),abs(nx_vect2[i]-nx_vect2[i-2]),abs(ny_vect2[i]-ny_vect2[i-2])]))
    alt_vect2=np.zeros([4])
    alt_vect2[0]=np.pi-thetavect2[i]
    alt_vect2[1]=-nx_vect2[i]
    alt_vect2[2]=-ny_vect2[i]
    alt_vect2[3]=-nz_vect2[i]
    diff_alt_1=np.amax(np.array([abs(alt_vect2[0]-thetavect2[i-1]),abs(alt_vect2[1]-nx_vect2[i-1]),abs(alt_vect2[2]-ny_vect2[i-1])]))
    diff_alt_2=np.amax(np.array([abs(alt_vect2[0]-thetavect2[i-2]),abs(alt_vect2[1]-nx_vect2[i-2]),abs(alt_vect2[2]-ny_vect2[i-2])]))
    if min(diff_alt_1,diff_alt_2)<min(diff1,diff2):
        thetavect2[i]=alt_vect2[0]
        nx_vect2[i]=alt_vect2[1]
        ny_vect2[i]=alt_vect2[2]
        nz_vect2[i]=alt_vect2[3]

In [ ]:
mat_theta=thetavect2.reshape((num_pix,num_pix))
mat_nx=nx_vect2.reshape((num_pix,num_pix))
mat_ny=ny_vect2.reshape((num_pix,num_pix))
mat_nz=nz_vect2.reshape((num_pix,num_pix))

In [ ]:
#minimum fidelity between neighbouring pixels for the 2nd run
tol2=0.98

In [ ]:
#correcting wrong pixels by interpolating between their neighbours (again)

for i in range(num_pix):
    for j in range(num_pix):
    
        if close_fid(i,j)<tol2:
            mat_theta[i,j]=inter_NN(i,j,mat_theta)
            mat_nx[i,j]=inter_NN(i,j,mat_nx)
            mat_ny[i,j]=inter_NN(i,j,mat_ny)
            mat_nz[i,j]=inter_NN(i,j,mat_nz)

In [ ]:
alt_mat_theta=np.ones([num_pix,num_pix])*np.pi - mat_theta
alt_mat_nx=-mat_nx
alt_mat_ny=-mat_ny
alt_mat_nz=-mat_nz

In [ ]:
for j in range(num_pix):
    for i in range(num_pix):
    
        if alt_close_jump(-i,-j)<close_jump(-i,-j):
            mat_theta[-i,-j]=alt_mat_theta[-i,-j]
            mat_nx[-i,-j]=alt_mat_nx[-i,-j]
            mat_ny[-i,-j]=alt_mat_ny[-i,-j]
            mat_nz[-i,-j]=alt_mat_nz[-i,-j]

In [ ]:
#if total inversion is needed

mat_theta=np.ones([num_pix,num_pix])*np.pi - mat_theta
mat_nx=-mat_nx
mat_ny=-mat_ny
mat_nz=-mat_nz

Plot comparison

In [ ]:
xvals=np.arange(0,num_pix)
yvals=np.arange(0,num_pix)
y,x=np.meshgrid(xvals,yvals)

In [ ]:
plt.figure(1)
fig, ax = plt.subplots(figsize=(12, 8))

c = ax.pcolormesh(x, y, mat_theta[:-1, :-1], cmap='RdBu', vmin=mat_theta.min(), vmax=mat_theta.max())
ax.set_title('Theta plot (net)')
# set the limits of the plot to the limits of the data
ax.axis([x.min(), x.max(), y.min(), y.max()])
plt.xlabel('x',fontsize=18)
plt.ylabel('y',fontsize=18)
cbar=fig.colorbar(c, ax=ax)
cbar.set_label('Theta',fontsize=18)

plt.figure(2)

fig, ax = plt.subplots(figsize=(12, 8))

c = ax.pcolormesh(x, y, mat_nx[:-1, :-1], cmap='RdBu', vmin=mat_nx.min(), vmax=mat_nx.max())
ax.set_title('nx plot (net)')
# set the limits of the plot to the limits of the data
ax.axis([x.min(), x.max(), y.min(), y.max()])
plt.xlabel('x',fontsize=18)
plt.ylabel('y',fontsize=18)
cbar=fig.colorbar(c, ax=ax)
cbar.set_label('nx',fontsize=18)

plt.figure(3)
fig, ax = plt.subplots(figsize=(12, 8))

c = ax.pcolormesh(x, y, mat_ny[:-1, :-1], cmap='RdBu', vmin=mat_ny.min(), vmax=mat_ny.max())
ax.set_title('ny plot (net)')
# set the limits of the plot to the limits of the data
ax.axis([x.min(), x.max(), y.min(), y.max()])
plt.xlabel('x',fontsize=18)
plt.ylabel('y',fontsize=18)
cbar=fig.colorbar(c, ax=ax)
cbar.set_label('ny',fontsize=18)

plt.figure(4)
fig, ax = plt.subplots(figsize=(12, 8))

c = ax.pcolormesh(x, y, mat_nz[:-1, :-1], cmap='RdBu', vmin=mat_nz.min(), vmax=mat_nz.max())
ax.set_title('nz plot (net)')
# set the limits of the plot to the limits of the data
ax.axis([x.min(), x.max(), y.min(), y.max()])
plt.xlabel('x',fontsize=18)
plt.ylabel('y',fontsize=18)
cbar=fig.colorbar(c, ax=ax)
cbar.set_label('nz',fontsize=18)

plt.show()

# Fidelity (if theoretical values are available)

In [ ]:
#the fidelity doesn't care if it's U or -U
Fvals=np.zeros([num_pix,num_pix])
for i in range(num_pix):
    for j in range(num_pix):
        netU=op_par(mat_theta[i,j],mat_nx[i,j],mat_ny[i,j],mat_nz[i,j])
        thU=op_par(theta_th_mat[i,j],nx_th_mat[i,j],ny_th_mat[i,j],nz_th_mat[i,j])
        Fvals[i,j]=fidelity(netU,thU)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

c = ax.pcolormesh(x, y, Fvals[:-1, :-1], cmap='RdBu', vmin=Fvals.min(), vmax=Fvals.max())
ax.set_title('Fidelity plot')
# set the limits of the plot to the limits of the data
ax.axis([x.min(), x.max(), y.min(), y.max()])
plt.xlabel('x',fontsize=18)
plt.ylabel('y',fontsize=18)
cbar=fig.colorbar(c, ax=ax)
cbar.set_label('F',fontsize=18)

plt.show()

In [ ]:
meanF=np.mean(Fvals)
meanF